<a href="https://colab.research.google.com/github/vedantdave77/project.Orca/blob/master/Project/project-Dog_Breed_CNN_Classification/CNN_Classifier(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN From Scratch 

---

### Embeding Google Drive for Data Access



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# set parameters which I usually used during project 
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
img_short_side_resize = 256
img_input_size = 224
shuffle = True
num_workers = 16
batch_size = 64

## Data : [Access/ Transform/ Load]

Reference: 
1. [Data Loader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

2. [Customize Data Set Operation](https://pytorch.org/docs/stable/torchvision/datasets.html)

3. [Data Transform](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)


In [14]:
# import required libraries 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision.transforms as transforms
from torchvision import datasets 

# define data_transformation and batch_size 
transform_train = transforms.Compose([
                                      transforms.Resize(img_short_side_resize),
                                      transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation =0.2, hue = 0.1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomResizedCrop(img_input_size, scale=(0.08,1), ratio = (1,1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = norm_mean, std = norm_std)
                                       ])

transform_test = transforms.Compose([
                                     transforms.Resize(img_input_size),
                                     transforms.FiveCrop(img_input_size),
                                     transforms.Lambda(lambda crops: torch.stack([
                                                                                  transforms.Compose([
                                                                                                      transforms.ToTensor(),
                                                                                                      transforms.Normalize(mean = norm_mean, std = norm_std)])(crop) for crop in crops
                                                                                  ]))

                                    ])

# load data (define datasets)
train_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/train/",transform_train)
valid_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/valid/",transform_test)
test_data  = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/test/",transform_test)

# separate imput and labels(classes)
data = {"train" : train_data, "valid" : valid_data, "test" : test_data}
n_classes = len(train_data.classes)

# create loaders (train, valid, test)
train_loader = torch.utils.data.DataLoader(data["train"], batch_size = batch_size, num_workers = num_workers, shuffle = shuffle, pin_memory = True)
valid_loader = torch.utils.data.DataLoader(data["valid"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True) 
test_loader = torch.utils.data.DataLoader(data["valid"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True)

# loader dictionary
loaders_dict = {"train" : train_loader, "valid" : valid_loader, "test" : test_loader}

## Model Architecture :
### Create CNN Classifier 


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# generate the CNN Architecture from Scratch
class CNN(nn.module):
    def __init__(self,n_classes,layer1_depth = 32):
        super(CNN, self).__init__()
        
        # define layer wise depth
        layer2_depth = layer1_depth *2                                         # 32 --> 64
        layer3_depth = layer2_depth *2                                         # 64 --> 128

        # define Max-pooling layer
        self.pool = nn.MaxPool2d(2,2)

        # convolution set 1 
        self.conv1_1 = nn.Conv2d(3, layer1_depth, 3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(layer1_depth, layer1_depth, 3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(layer1_depth, layer1_depth, 3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(layer1_depth)
        self.bn1_2 = nn.BatchNorm2d(layer1_depth)
        self.bn1_3 = nn.BatchNorm2d(layer1_depth)

        # convolution set 2 
        self.conv2_1 = nn.Conv2d(layer1_depth, layer2_depth, 3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(layer2_depth, layer2_depth, 3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(layer2_depth, layer2_depth, 3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(layer2_depth)
        self.bn2_2 = nn.BatchNorm2d(layer2_depth)
        self.bn2_3 = nn.BatchNorm2d(layer2_depth)

        # convolution set 3 
        self.conv3_1 = nn.Conv2d(layer2_depth, layer3_depth, 3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(layer3_depth, layer3_depth, 3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(layer3_depth, layer3_depth, 3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(layer3_depth)
        self.bn3_2 = nn.BatchNorm2d(layer3_depth)
        self.bn3_3 = nn.BatchNorm2d(layer3_depth)

        # output 
        self.output = nn.Linear(layer3_depth,n_classes)                         # 128 ---> 133

        # Initialize weight
        nn.init.kaiming_normal_(self.conv1_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv1_2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv1_3.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_3.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3_2.weight, nonlinearity='relu') 
        nn.init.kaiming_normal_(self.conv3_3.weight, nonlinearity='relu')       


""" 
Develop forward pass, with feature expandation and 
extraction by increase or maintain depthh of network. 

""" 

    def forward(self,x):
        # Conv Flow 1
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = F.relu(self.bn1_2(self.conv1_2(x)))
        x = F.relu(self.bn1_3(self.conv1_3(x)))
        x = self.pool(x)

        # Conv Flow 2
        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = F.relu(self.bn2_2(self.conv2_2(x)))
        x = F.relu(self.bn2_3(self.conv2_3(x)))
        x = self.pool(x)

        # Conv Flow 3
        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.bn3_2(self.conv3_2(x)))
        x = F.relu(self.bn3_3(self.conv3_3(x)))
        x = self.pool(x)

        # fuse the dimension (height=2, width =3)
        x = x.view(x.size(0),x.size(1),-1)
        x = x.max(2)[0]

        # output 
        x = self.output(x)
        
        return x

In [ ]:
model_scratch = CNN(n_classes)

if use_cuda:
    model_scratch.cuda()           